<a href="https://colab.research.google.com/github/pedrov718/Content-Based-Book-Rec-System/blob/main/book_reccomendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import spacy.cli
spacy.cli.download("en_core_web_lg")

import re
import string
import nltk
import numpy as np

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import data_table
data_table.enable_dataframe_formatter()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs.csv')

In [ ]:
df.head()

In [ ]:
df['Blurb'][10:15]

In [ ]:
# Load spacy
nlp = spacy.load('en_core_web_lg')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    #Remove stop words
    text = text.split()

    #Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', word) for word in text]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string


In [ ]:
print("Cleaning train data...\n")

df["blurb_clean"] = df['Blurb'].apply(lambda x: clean_string(x, stem='Spacy'))

In [ ]:
df.head(2)

In [ ]:
df['Title_lower'] = df['Title'].apply(lambda x: x.lower())

In [ ]:
df['blurb_clean_title'] =  df['Title_lower'] +  ' ' + df['blurb_clean']

In [ ]:
df['blurb_clean_title'][10]

In [3]:
import pandas

In [4]:
df.to_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs_clean.csv')

NameError: ignored

### Run locally here

In [14]:
df = pd.read_csv('/content/drive/MyDrive/Book_Rec/books_with_blurbs_clean.csv')

#### Now that we have the data cleaned and saved to Drive we can use tfidf to convert our text data to numerical

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['blurb_clean_title'])

In [17]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index])

In [27]:
most_similiar_book()

Write a line from your favorite book? baos



 Title: 
 Decision in Normandy 

 Description: 
 Here, for the first time in paperback, is an outstanding military history that offers a dramatic new perspective on the Allied campaign that began with the invasion of the D-Day beaches of Normandy. Nationa advertising in Military History. 

 About the Author: 
 Sorry we dont have any information about Carlo D'Este at the moment None


#### TOP N Number of book reccomendations

In [19]:
def top_n_similiar_books():
  
  n = int(input("How many books would you like reccomended? Please input a number. "))

  sentence = [input("Write the title or line from your favorite book? ")]
  
  y = vectorizer.transform(sentence)
  
  cosine_sim = cosine_similarity(X,y)

  # Get the pairwsie similarity scores
  sim_scores = list(enumerate(cosine_sim))
  # Sort the books based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # Get the scores for n most similar books
  sim_scores = sim_scores[0:n]
  
  titles = []
  author = []
  desc = []

  for pair in sim_scores:
    indx, score = pair
    titles.append(df["Title"][indx])
    author.append(df["Author"][indx])
    desc.append(df["Blurb"][indx])
  
  reccs = pd.DataFrame(list(zip(titles, author, desc)), columns =['titles', 'author', 'blurbs'])
  
  return reccs

In [20]:
top_n_similiar_books()

How many books would you like reccomended? Please input a number. 4
Write the title or line from your favorite book? baos


,titles,author,blurbs
0,Decision in Normandy,Carlo D'Este,"Here, for the first time in paperback, is an o..."
1,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,"The fascinating, true story of the world's dea..."
2,The Kitchen God's Wife,Amy Tan,Winnie and Helen have kept each others worst s...
3,What If?: The World's Foremost Military Histor...,Robert Cowley,Historians and inquisitive laymen alike love t...


In [21]:
!pip install git+https://github.com/internetarchive/openlibrary-client.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/internetarchive/openlibrary-client.git to /tmp/pip-req-build-u9f93avx
  Running command git clone --filter=blob:none --quiet https://github.com/internetarchive/openlibrary-client.git /tmp/pip-req-build-u9f93avx
  Resolved https://github.com/internetarchive/openlibrary-client.git to commit 2367838042ccda2e0ddd16ed75b2e2cafac016b6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [22]:
from olclient.openlibrary import OpenLibrary
ol = OpenLibrary()
author_olid = ol.Author.get_olid_by_name('Dan Brown')
author_obj = ol.get(author_olid)

In [23]:
from IPython.display import display, HTML

In [29]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  author_olid = ol.Author.get_olid_by_name(df['Author'].iloc[max_index])
  if ol.get(author_olid) != None:
    author_obj = ol.get(author_olid)
  else:
    print('Sorry author is not in our database')
  if author_obj.bio != None:
    author_desc = author_obj.bio.split('\r\n\r\n')[0]
  else:
    author_desc = f"Sorry we dont have any information about {df['Author'].iloc[max_index]} at the moment"
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index], '\n\n', 'About the Author:', '\n', author_desc, 
               display(HTML('<img src="https://covers.openlibrary.org/b/isbn/' + df['ISBN'].iloc[max_index] + '-L.jpg" />')))

In [37]:
most_similiar_book()

Write a line from your favorite book? a murder of crows



 Title: 
 A Murder of Crows 

 Description: 
 Ingrid Langley had endured a divorce from her husband, Patrick, and never intended to see him again. But Patrick was desperate; as an agent for British Intelligence, he needed a woman to be his cover. And how could Ingrid refuse--when he was investigating the murder of her second husband? Martin's. 

 About the Author: 
 Sorry we dont have any information about Margaret Duffy at the moment None
